In [ ]:
from datetime import datetime
import itertools
import json
import math
import os
import sys

import importlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import scipy
from scipy.spatial import KDTree
import seaborn as sns
from sklearn import metrics
import torch
from torch.utils.data import DataLoader

from utils import data_utils, data_loader, shape_utils

importlib.reload(data_utils)
importlib.reload(data_loader)
importlib.reload(shape_utils)

from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

RUN_FOLDER = "../results/debug/"

In [ ]:
kcm_inputs = data_utils.load_all_inputs(RUN_FOLDER, "kcm/")
atb_inputs = data_utils.load_all_inputs(RUN_FOLDER, "atb/")

In [ ]:
# Sample shingle ids and choose dist/time ranges to test
shingle_ids = pd.unique(kcm_inputs['test_traces']['shingle_id'])
shingle_id_sample = np.random.choice(shingle_ids, 100)
d_buffers = np.linspace(1,10000,20)
t_buffers = np.linspace(1,60*60*6,20)
# Test shingle data against all other data
traces = kcm_inputs['test_traces']
shingle_traces = traces[traces['shingle_id'].isin(shingle_id_sample)][['x','y','locationtime','speed_m_s','shingle_id']]
shingle_groups = shingle_traces.groupby('shingle_id')
adj_traces = traces[~traces['shingle_id'].isin(shingle_id_sample)][['x','y','locationtime','speed_m_s']].values
# Save results for each shingle, and buffer combination
targets_res = np.zeros((len(shingle_id_sample), len(d_buffers), len(t_buffers)))
preds_res = np.zeros((len(shingle_id_sample), len(d_buffers), len(t_buffers)))

In [ ]:
# Run adjacent analysis for each d and t buffer
for d, dist_buffer in enumerate(d_buffers):
    print(d)
    for t, t_buffer in enumerate(t_buffers):
        print(t)
        res = shingle_groups.apply(shape_utils.get_adjacent_metric, adj_traces, dist_buffer, t_buffer)
        targets = np.array([x[0] for x in res])
        preds = np.array([x[1] for x in res])
        targets[np.isnan(targets)] = np.mean(targets[~np.isnan(targets)])
        preds[np.isnan(preds)] = np.mean(preds[~np.isnan(preds)])
        targets_res[:,d,t] = targets
        preds_res[:,d,t] = preds

In [ ]:
# Rows, Columns
r_values = np.empty((preds.shape[1], preds.shape[2]))
for d in range(preds.shape[1]):
    for t in range(preds.shape[2]):
        R2, x_plt, y_plt = shape_utils.fit_poly(adj[:,d,t], target)
        r_values[d,t] = R2
r_values

In [ ]:
fig = px.imshow(r_values, labels={"x":"Time","y":"Dist"})
fig.show()